## Semantic Kernel: Ramp-Up based on SK's Documentation

To get the latest version of SK Python package, use:

``` bash
pip install --upgrade semantic-kernel
```

## 📒 Notebook 4: Observability

### 🪜 Step 1: Configure environment

In [1]:
# Import required packages
import asyncio
import logging
import os

# OpenTelemetry imports for Azure Monitor
from azure.monitor.opentelemetry.exporter import (
    AzureMonitorLogExporter,
    AzureMonitorMetricExporter,
    AzureMonitorTraceExporter,
)
from opentelemetry._logs import set_logger_provider
from opentelemetry.metrics import set_meter_provider
from opentelemetry.sdk._logs import LoggerProvider, LoggingHandler
from opentelemetry.sdk._logs.export import BatchLogRecordProcessor
from opentelemetry.sdk.metrics import MeterProvider
from opentelemetry.sdk.metrics.export import PeriodicExportingMetricReader
from opentelemetry.sdk.metrics.view import DropAggregation, View
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.semconv.resource import ResourceAttributes
from opentelemetry.trace import set_tracer_provider

# Semantic Kernel imports
from semantic_kernel import Kernel
from semantic_kernel.contents import ChatHistory
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.azure_ai_inference import AzureAIInferenceChatCompletion

# Azure AI Inference imports
from azure.ai.inference.aio import ChatCompletionsClient
from semantic_kernel.connectors.ai.azure_ai_inference import AzureAIInferenceChatPromptExecutionSettings
from azure.identity.aio import DefaultAzureCredential


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# Set Azure OpenAI backend variables
AOAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_API_DEPLOY")
AOAI_ENDPOINT = os.getenv("AZURE_OPENAI_API_BASE")
AOAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_APPINSIGHTS = os.getenv("AZURE_APPINSIGHTS_CONNSTRING")

### 🪜 Step 2: Set up OpenTelemetry for Logging, Tracing and Metrics

In [4]:
# Create App Insights resource to represent SK application
resource = Resource.create({ResourceAttributes.SERVICE_NAME: "semantic-kernel-observability-notebook"})

print(f"Application Insights Connection String: '{AZURE_APPINSIGHTS[:15]}...' (truncated for display)")
print(f"Service Resource Name: '{resource.attributes[ResourceAttributes.SERVICE_NAME]}'")

Application Insights Connection String: 'Instrumentation...' (truncated for display)
Service Resource Name: 'semantic-kernel-observability-notebook'


In [5]:
# Helper functions to send telemetry to Azure App Insights
def set_up_logging():
    """Configures OpenTelemetry for logging."""
    exporter = AzureMonitorLogExporter(connection_string=AZURE_APPINSIGHTS)
    logger_provider = LoggerProvider(resource=resource)
    logger_provider.add_log_record_processor(BatchLogRecordProcessor(exporter))
    set_logger_provider(logger_provider)

    handler = LoggingHandler()
    # Filter to only process records from semantic_kernel
    handler.addFilter(logging.Filter("semantic_kernel"))
    logger = logging.getLogger()
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)
    print("  - Logging setup complete.")

def set_up_tracing():
    """Configures OpenTelemetry for tracing."""
    exporter = AzureMonitorTraceExporter(connection_string=AZURE_APPINSIGHTS)
    tracer_provider = TracerProvider(resource=resource)
    tracer_provider.add_span_processor(BatchSpanProcessor(exporter))
    set_tracer_provider(tracer_provider)
    print("  - Tracing setup complete.")

def set_up_metrics():
    """Configures OpenTelemetry for metrics."""
    exporter = AzureMonitorMetricExporter(connection_string=AZURE_APPINSIGHTS)
    meter_provider = MeterProvider(
        metric_readers=[PeriodicExportingMetricReader(exporter, export_interval_millis=5000)],
        resource=resource,
        views=[
            # Drop all instrument names except for those starting with "semantic_kernel"
            View(instrument_name="*", aggregation=DropAggregation()),
            View(instrument_name="semantic_kernel*"),
        ],
    )
    set_meter_provider(meter_provider)
    print("  - Metrics setup complete.")

In [6]:
# Enable telemetry
set_up_logging()
set_up_tracing()
set_up_metrics()

  - Logging setup complete.
  - Tracing setup complete.
  - Metrics setup complete.


### 🪜 Step 3: Add AOAI Chat Completion service

In [7]:
# Initialise kernel
kernel = Kernel()

In [8]:
# Configure logging
logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

In [9]:
# Add Azure OpenAI chat completion
chat_completion = AzureChatCompletion(
    deployment_name = AOAI_DEPLOYMENT,
    endpoint = AOAI_ENDPOINT,
    api_version = AOAI_API_VERSION,
    service_id = "azure_openai_chat",
)

kernel.add_service(chat_completion)

### 🪜 Step 4: Invoke a prompt and observe telemetry in App Insights

After execution of this cell, you should be able to find traces and dependency logging in Azure App Insights (under Investigate -> Transaction Search).

In [10]:
# Invoke a simple prompt
try:
    answer = await kernel.invoke_prompt("Tell me an interesting fact about red pandas in one sentence.")
    print(f"AI Answer: {answer}")
except Exception as e:
    print(f"An error occurred during prompt invocation: {e}")

AI Answer: Red pandas are more closely related to raccoons than to giant pandas, despite their name and similar appearance.


### 🪜 Step 5: Add Azure AI Inference service

In [16]:
# Define AI Inference chat completion
# NOTE: Ensure your Azure user identity has "Cognitive Services OpenAI Contributor" role 
# assigned on the Azure OpenAI resource for DefaultAzureCredential to work
chat_completion_service = AzureAIInferenceChatCompletion(
    ai_model_id = AOAI_DEPLOYMENT,
    client = ChatCompletionsClient(
        api_version = AOAI_API_VERSION,
        endpoint = f"{str(AOAI_ENDPOINT).strip('/')}/openai/deployments/{AOAI_DEPLOYMENT}",
        credential = DefaultAzureCredential(),
        credential_scopes = ["https://cognitiveservices.azure.com/.default"],
    ),
)

In [12]:
# Add AI Inference service to the kernel
kernel.add_service(chat_completion_service)

### 🪜 Step 6: Invoke a prompt and observe telemetry in Azure AI Foundry

In [13]:
# Define execution settings
execution_settings = AzureAIInferenceChatPromptExecutionSettings()

In [14]:
# Initialise chat history
chat_history = ChatHistory()
chat_history.add_user_message("Tell me an interesting fact about red pandas in one sentence.")

In [17]:
# Non-streaming chat completion
response = await chat_completion_service.get_chat_message_content(
    chat_history = chat_history,
    settings = execution_settings,
)

print(f"AI Inference Answer: {response}")

AI Inference Answer: Red pandas are not closely related to giant pandas; instead, they belong to their own unique family called Ailuridae, and they share more genetic similarities with raccoons and weasels.
